In [1]:
# Importing necessary dependencies
import pandas as pd
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
import os


In [2]:
# Data Extraction
try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted successfully!')

except Exception as e:
    print(f'An error occurred: {e}')

Data Extracted successfully!


In [4]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [6]:
# Data cleaning and transformation
# remove duplicates, i.e remove rows that are completely duplicate
data.drop_duplicates(inplace=True)

In [8]:
# Handle missing values numeric columns...
# ...we do this by filling missing numeric values with the mean or median
numeric_columns = data.select_dtypes(include= ['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col : data[col].mean()}, inplace=True)

In [10]:
#Handle missing values of string/object columns...
# ...we do this by filling missing string/object values with 'unknown'
string_columns = data.select_dtypes(include = ['object']).columns
for col in string_columns:
    data.fillna({col : 'Unknown'}, inplace=True)

In [54]:
# Converting Date to a DateTime data type
data['Date'] = pd.to_datetime(data['Date'])

In [11]:
# Checking to see if duplicates have been removed
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [12]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [35]:
# Create the product table
product = data[['ProductName']].drop_duplicates().reset_index(drop=True)
product.index.name = 'ProductID'
product = product.reset_index()

In [36]:
product.head(30)

,ProductID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni
5,5,Pizza Margherita
6,6,Pizza Beef Double decker
7,7,Cheesecake
8,8,Fruit Cake


In [37]:
# Create the customers Table
customer = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
                 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customer.index.name = 'CustomerID'
customer = customer.reset_index()

In [38]:
customer.head()

,CustomerID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [39]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerID            1000 non-null   int64 
 1   CustomerName          1000 non-null   object
 2   CustomerAddress       1000 non-null   object
 3   Customer_PhoneNumber  1000 non-null   object
 4   CustomerEmail         1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [40]:
# Staff Table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()

In [41]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   StaffID      1000 non-null   int64 
 1   Staff_Name   1000 non-null   object
 2   Staff_Email  1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [49]:
# Transaction table
transaction = data.merge(product, on=['ProductName'], how='left')\
                  .merge(customer, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left')\
                  .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left')

transaction.index.name = 'TransactionID'
transaction = transaction.reset_index()\
                         [['Date', 'TransactionID', 'ProductID', 'CustomerID', 'StaffID', 'Quantity', 'UnitPrice', 'StoreLocation',
                            'PaymentType', 'PromotionApplied', 'Weather', 'Temperature', 'StaffPerformanceRating', 'CustomerFeedback',
                            'DeliveryTime_min', 'OrderType', 'DayOfWeek', 'TotalSales']]

In [52]:
transaction.head()

,Date,TransactionID,ProductID,CustomerID,StaffID,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,0,0,0,0,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,Sunday,25.064608
1,2023-01-01 01:00:00,1,1,1,1,1,7.083070,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,Sunday,7.083070
2,2023-01-01 02:00:00,2,2,2,2,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,Unknown,43,Phone Order,Sunday,33.680321
3,2023-01-01 03:00:00,3,3,3,3,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,Sunday,14.629647
4,2023-01-01 04:00:00,4,4,4,4,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,Sunday,7.577727


In [53]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   TransactionID           1000 non-null   int64         
 2   ProductID               1000 non-null   int64         
 3   CustomerID              1000 non-null   int64         
 4   StaffID                 1000 non-null   int64         
 5   Quantity                1000 non-null   int64         
 6   UnitPrice               1000 non-null   float64       
 7   StoreLocation           1000 non-null   object        
 8   PaymentType             1000 non-null   object        
 9   PromotionApplied        1000 non-null   bool          
 10  Weather                 1000 non-null   object        
 11  Temperature             1000 non-null   float64       
 12  StaffPerformanceRating  1000 non-null   object   

In [56]:
# Save data as csv files
data.to_csv('Clean_dataset\clean_data.csv', index=False)
product.to_csv('Clean_dataset\product.csv', index=False)
customer.to_csv('Clean_dataset\customer.csv', index=False)
staff.to_csv('Clean_dataset\staff.csv', index=False)
transaction.to_csv('Clean_dataset/transaction.csv', index=False)

In [57]:
# Data Loading
# Loading the environment variables from .env files
load_dotenv()
connect_str = os.getenv('AZURE_CONNECT_STR')
container_name = os.getenv('CONTAINER_NAME')

In [58]:
# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

In [60]:
# Load data to Azure Blob Storage
files = [
    (data, 'cleandataset/cleaned_zipco_foods_data.csv'),
    (product, 'cleandataset/product.csv'),
    (customer, 'cleandataset/customer.csv'),
    (staff, 'cleandataset/staff.csv'),
    (transaction, 'cleandataset/transaction.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')

cleandataset/cleaned_zipco_foods_data.csv loaded into Azure Blob Storage
cleandataset/product.csv loaded into Azure Blob Storage
cleandataset/customer.csv loaded into Azure Blob Storage
cleandataset/staff.csv loaded into Azure Blob Storage
cleandataset/transaction.csv loaded into Azure Blob Storage
